# Scraping MPs' register of interests

In [0]:
#!/usr/bin/env python
# encoding=utf8  - lines below from https://stackoverflow.com/questions/21129020/how-to-fix-unicodedecodeerror-ascii-codec-cant-decode-byte
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

#line above because of £ sign and https://www.python.org/dev/peps/pep-0263/
import scraperwiki
import requests
import lxml.html

import re
def find_largest_num(stringtore):
    #examples could be £60, £107.57, £1,155.88 but not '72 Broad Street'
    #abandoned regex as grabbed whole line and split equally effective
    #regex = r"£.*[ \t\n\r\f\v]"
#    p = re.compile('£.*[ \t\n\r\f\v]')
 #   rematches = re.findall(regex, "dinner £1,155.88 plus flights at an estimated value of £1,405, total £2,560.88")
  #  print len(rematches)
   # for match in rematches:
    #    print 'matched:', match
#    rematches = re.findall(regex, stringtore)
 #   print len(rematches)
  #  for match in rematches:
   #     print 'matched:', match
    splitstring = stringtore.replace("\n"," ").split("£")
    numbers = len(splitstring)
    print "there are ", numbers, "mentions of £"
    biggestnum = 0
    totalnums = 0
    if len(splitstring)>1:
        print "first figure: ", splitstring[1].split(" ")[0]
        storingnums = []
        for i in range(1,numbers):
            thisnum = splitstring[i].split(" ")[0].replace(",","").replace(")","").replace("-","").replace("p.a","").replace("2.676.41","2676.41").replace("1.261.79","1261.79")
            print thisnum
            #print repr(thisnum)
            #print type(thisnum)
            if thisnum.endswith('.'):
                thisnum = thisnum[:-1]
            storingnums.append(float(thisnum.strip()))
            print storingnums
        biggestnum = max(storingnums)
        totalnums = len(storingnums)
    print biggestnum
    return biggestnum, totalnums
        
stringexample = '''Fees received for articles written for The Guardian. Address: Guardian News & Media, Kings Place, 90 York Way, London N1 9GU: 3 November 2016, received £60. Hours: 30 mins. (Registered 10 April 2017) 10 November 2016, received £100. Hours: 1 hr. (Registered 10 April 2017)
22 December 2016, received £285. Hours: 2.5 hrs. (Registered 10 April 2017)
23 February 2017, received £410. Hours: 6.5 hrs. (Registered 10 April 2017)
16 March 2017, received £100. Hours: 1 hr. (Registered 10 April 2017)
dinner £1,155.88 plus flights at an estimated value of £1,405, total £2,560.88
Destination of visit: Nigeria'''
#biggestnum, totalnums = find_largest_num(stringexample)
#print biggestnum, totalnums


# We can assume the new interest URL will be http://www.publications.parliament.uk/pa/cm/cmregmem/contents1718.htm

#example URL: https://www.publications.parliament.uk/pa/cm/cmregmem/170502/hammond_philip.htm
indexurl = "http://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/parliamentary-commissioner-for-standards/registers-of-interests/register-of-members-financial-interests/"
baseurl = "https://www.publications.parliament.uk/pa/cm/cmregmem/"

#This function starts from the register of interests index and grabs the latest year
def findlatestyrurl(indexurl):
    html = requests.get(indexurl).content
    root = lxml.html.fromstring(html)
    ##ctl00_ctl00_FormContent_SiteSpecificPlaceholder_PageContent_ctlMainBody_wrapperDiv
    links = root.cssselect('div#ctl00_ctl00_FormContent_SiteSpecificPlaceholder_PageContent_ctlMainBody_wrapperDiv li a')
    print '--LATEST REGISTER YEAR TITLE IS--', links[0].text_content()
    print '--LATEST REGISTER YEAR URL IS--',links[0].attrib['href']
    #return links[1].attrib['href'] #- for 2016-17 etc
    return links[0].attrib['href']

#This function starts from the year page and grabs the latest register for that year
def findlatestregister(indexurl):
    latesturl = findlatestyrurl(indexurl)
    html = requests.get(latesturl).content
    print 'chars 400-1000 of HTML: ', html[400:1000]
    root = lxml.html.fromstring(html)
    # link is within div id="maincontent", then within a table cell with a link
    links = root.cssselect('div#maincontent tr td a')
    if latesturl.split('/')[-1] == "contents1719.htm":
        print '--LATEST REGISTER TEXT IS--',links[2].text_content()
        print '--LATEST REGISTER DATE URL IS--', links[2].attrib['href']
        latestregisterurl = links[2].attrib['href']
    #the link to the latest register is typically the fourth
    else:
        print '--LATEST REGISTER TEXT IS--',links[3].text_content()
        print '--LATEST REGISTER DATE URL IS--', links[3].attrib['href']
        latestregisterurl = links[3].attrib['href']
    return latestregisterurl

#This function starts from a specific register and grabs all MP links
def grabmplinks(latestregister):
    print 'GRABBING MP LINKS--', latestregister
    html = requests.get(baseurl+latestregister).content
    root = lxml.html.fromstring(html)
    mps = root.cssselect('p a')
    if latestregister.split('/')[-2] == "170707":
    #The first 24 results are not MPs, but the 25th should be Diane Abbott
        firstmp = 25 # In 2017-19 it's one link later
        print 'FIRST MP (LINK 26) IS: ', mps[firstmp].text_content()
    elif latestregister.split('/')[-2] == "170731":
    #The first 24 results are not MPs, but the 25th should be Diane Abbott
        firstmp = 25 # In 2017-19 it's one link later
        print 'FIRST MP (LINK 26) IS: ', mps[firstmp].text_content() 
    elif latestregister.split('/')[-2] == "170829":
    #The first 24 results are not MPs, but the 25th should be Diane Abbott
        firstmp = 25 # In 2017-19 it's one link later
        print 'FIRST MP (LINK 26) IS: ', mps[firstmp].text_content() 
    else:
        firstmp = 24
        print 'FIRST MP (LINK 25) IS: ', mps[firstmp].text_content()
    return mps[firstmp:]

def scrapemppage(url, latestregister):
    urltoscrape = baseurl+latestregister.replace('part1','').replace('contents.htm',url)
    print '--SCRAPING--', urltoscrape
    html = requests.get(urltoscrape).content
    root = lxml.html.fromstring(html)
    h2s = root.cssselect('h2')
    mpname = h2s[0].text_content()
    print "MP name and constituency is ", mpname
    constituency = mpname.split("(")[1].replace(")","")
    textblocks = root.cssselect('#mainTextBlock')
    print 'there are ', len(textblocks), 'blocks'
    print 'there are ', len(textblocks), 'blocks'
    #splitting on each category heading is most error-free given inconsistent HTML
    #this category uses 'portfolio' in 2017 but not in 2016
    splitting = textblocks[0].text_content().split('. Land and property') # category 6 after June 2015, category 8 before
    splittingjobs = textblocks[0].text_content().split('1. Employment and earnings')
    splitting2 = textblocks[0].text_content().split('Support linked to an MP but received by a local party organisation or indirectly via a central party organisation')
    splitting2b = textblocks[0].text_content().split('Any other support not included in Category 2(a)')
    splittinggifts3 = textblocks[0].text_content().split('3. Gifts')
    splittinggifts5 = textblocks[0].text_content().split('5. Gifts')
    splitting_shares = textblocks[0].text_content().split('7. (i) Shareholdings')
    splitting_misc = textblocks[0].text_content().split('8. Miscellaneous')
    splitting_fam = textblocks[0].text_content().split('9. Family')
    splitting_famlobbying = textblocks[0].text_content().split('10. Family')
    print len(splitting)
    #These are default entries which remain only if no match is found
    jobs = 'No entry'
    support2 = 'No entry'
    support2b = 'No entry'
    gifts3 = 'No entry'
    gifts5 = 'No entry'
    propertyentry = '   No entry'
    miscentry = 'No entry'
    sharescentry = 'No entry'
    famlobbying = 'No entry'
    fam = 'No entry'
    #These test if the page contains particular categories, and stores relevant text if so
    if '1. Employment and earnings' in textblocks[0].text_content():
        jobs = splittingjobs[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0].split('. Land and property')[0].split('5. Gifts')[0].split('4. Visits')[0].split('3. Gifts')[0].split('2. (a) ')[0].split('2. (b)')[0]
    if "3. Gifts" in textblocks[0].text_content():
        gifts3 = splittinggifts3[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0].split('. Land and property')[0].split('5. Gifts')[0].split('4. Visits')[0]
        print gifts3
    if "5. Gifts" in textblocks[0].text_content():
        gifts5 = splittinggifts5[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0].split('. Land and property')[0]
        print gifts5
    if "Support linked to an MP but received by a local party organisation or indirectly via a central party organisation" in textblocks[0].text_content():
        support2 = splitting2[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0].split('. Land and property')[0].split('5. Gifts')[0].split('4. Visits')[0].split('3. Gifts, benefits')[0].split('2. (a) ')[0].split('2. (b)')[0]
        print support2
    if "Any other support not included in Category 2(a)" in textblocks[0].text_content():
        support2b = splitting2b[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0].split('. Land and property')[0].split('5. Gifts')[0].split('4. Visits')[0].split('3. Gifts, benefits')[0]
        print support2b
    #this category uses 'portfolio' in 2017 but not in 2016. In march 2015 it is category 8 not 6, which throws an error
    if "Land and property" in textblocks[0].text_content():
        propertyentry = splitting[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0]
        #trying to write an alternative for March 2015 which grabs property as category 8
        #ff = textblocks[0].text_content().split('8. Land and property')
        #propertyentry = ff[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('7. (i) Shareholdings')[0].split('Previous\n')[0]
        print propertyentry
    if "7. (i) Shareholdings" in textblocks[0].text_content():
        sharescentry = splitting_shares[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('8. Miscellaneous')[0].split('Previous\n')[0]
        print sharescentry
    #This category for 150330
    if "9. Registrable shareholdings" in textblocks[0].text_content():
        sharescentry = textblocks[0].text_content().split('9. Registrable shareholdings')[1].split('Previous\n')[0]
        print sharescentry
    if "8. Miscellaneous" in textblocks[0].text_content():
        miscentry = splitting_misc[1].split('9. Family members employed and paid from parliamentary expenses')[0].split('Previous\n')[0]
        print miscentry
    if "9. Family" in textblocks[0].text_content():
        fam = splitting_fam[1].split('Previous\n')[0].split('10. Family')[0]
        print fam
    if "10. Family" in textblocks[0].text_content():
        famlobbying = splitting_famlobbying[1].split('Previous\n')[0]
        print famlobbying
    return propertyentry, miscentry, sharescentry, support2, famlobbying, fam, gifts3, gifts5, jobs, support2b, mpname, constituency

latestregister = findlatestregister(indexurl)
#e.g. 170502/contents.htm
print '--FOUND LATEST REGISTER--', latestregister
#this next line can be uncommented to add data from another period
#latestregister = '160516/contents.htm'
#latestregister = '160606/contents.htm'
#latestregister = '170410/contents.htm'
#latestregister = '170502/contents.htm'
#latestregister = '170707/contents.htm'
#latestregister = "170731/contents.htm"
latestregister = "170829/contents.htm"
#latestregister = '150330/part1contents.htm' throws an error to do with mainTextBlock - 0 blocks in count
    #splitting = textblocks[0].text_content().split('. Land and property') 
#latestregister = "100927/part1contents.htm" throws an error:     
    #splitting2 = textblocks[0].text_content().split('Support linked to an MP but received by a local party organisation or indirectly via a central party organisation')
mps = grabmplinks(latestregister)
#create an empty record to store the results
record = {}
def getgifts(gifts3, gifts5, mpurl, mp):
    indexlist = []
    if gifts3 != "No entry":
        print "NOW GRABBING GIFTS: ", gifts3, gifts5, mpurl, mp
        gifttext = gifts3.replace(', benefits and hospitality from UK sources','')
        giftlist = gifttext.split("Name of donor: ")
        donorrecord = {}
        i = 0
        for gift in giftlist[1:]:
            i = i+1
            donorrecord['index'] = str(i)+mpurl
            print "GIFT: ", gift #ITV plcAddress of donor: The London Television Centre, Upper Ground, London SE1 9LTAmount of donation or nature and value if donation in kind: ticket to attend the National Television Awards with a value of £1,028Date received: 25 January 2017Date accepted: 25 January 2017Donor status: company, registration 4967001(Registered 13 March 2017)
            print "of ", len(giftlist), " gifts"
            donorrecord['donorname'] = gift.split("Address of donor: ")[0] #ITV plc
            try:
                donorrecord['donoraddress'] = gift.split("Address of donor: ")[1].split("Amount of donation or nature and value if donation in kind: ")[0] 
            except IndexError:
                donorrecord['donoraddress'] = 'could not extract'
            try:
                donationvalue = gift.split("donation in kind: ")[1].split("Date received: ")[0]
            except IndexError:
                print "TRYING AGAIN"
                donationvalue = gift.split(" amount of any donation): ")[1].split("Date received: ")[0]
            print "DONATIONVALUE", donationvalue
            donorrecord['donationdesc'] = donationvalue
            if len(donationvalue.split("total £"))>0:
                donorrecord['donationvalue'] = donationvalue.split("£")[1].replace(" ",";").replace(",","").replace("(","").replace(")","")
            else:
                donorrecord['donationvalue'] = 'could not extract'
            try:
                donorrecord['daterec'] = gift.split("Date received: ")[1].split("Date")[0]
            except IndexError:
                try:
                    donorrecord['daterec'] = gift.split("Date of receipt of donation: ")[1].split("Date")[0]
                except IndexError:
                    print "ERROR"
                    #donorrecord['daterec'] = 'could not extract
            try:
                donorrecord['dateacc'] = gift.split("Date accepted: ")[1].split("Donor status: ")[0]
            except IndexError:
                try:
                    donorrecord['dateacc'] = gift.split("Date of acceptance of donation: ")[1].split("Donor status: ")[0]
                except IndexError:
                    print "ERROR"
            try:
                donorrecord['donorstatus'] = gift.split("Donor status: ")[1]
            except IndexError:
                donorrecord['donorstatus'] = 'could not extract'
            #donorrecord['mp'] = mp
            donorrecord['url'] = mpurl
            #print "NAME: ", gift.split("Address of donor: ")[0] #ITV plc
            #print "ADDRESS: ", gift.split("Address of donor: ")[1].split("Amount of donation or nature and value if donation in kind: ")[0] 
            #print "DATEREC: ", 
            #print "DATEACC: ", 
            #print "DONORSTATUS: ", 
            print "DONORRECORD", donorrecord
            print "INDEX", donorrecord['index']
            indexlist.append(donorrecord['index'])
            print indexlist
            scraperwiki.sql.save(['index'],donorrecord, table_name="gifts")
    else:
        print "NO GIFTS TO GRAB"

#This is set at 2 for earlier pages, but should be 0 for new ones
startpoint = 0
for mp in mps[startpoint:]:
    mpurl = mp.attrib['href']
    print 'adding ', mpurl
    propertyentry, miscentry, sharescentry, support2, famlobbying, fam, gifts3, gifts5, jobs, support2b, mpname, constituency = scrapemppage(mpurl, latestregister)
    getgifts(gifts3, gifts5, mpurl, mp)
    record['MP'] = mp.text_content()
    record['MPname'] = mpname
    record['constituency'] = constituency
    #cleaning the heading out
    propertyentry = propertyentry.replace(': (i) value over ','').replace('100,000 and/or (ii) giving rental income of over ','').replace('10,000 a year','').replace('00,000 (ii) rental income of over ','')
    print 'propertyentry before cleaning:', propertyentry
    record['propertyentry'] = propertyentry[3:] #The index removes the first few characters, which are £ signs or spaces
    record['misc'] = miscentry
    record['employment and earnings'] = jobs
    record['emp_length'] = len(jobs)
    biggestnum, totalnums = 0,0
    if jobs != "No entry":
        biggestnum, totalnums = find_largest_num(jobs)
    record['emp_biggest'] = biggestnum
    record['emp_amounts'] = totalnums
    record['sharescentry'] = sharescentry
    record['support_linked'] = support2
    record['support_linked_COUNT'] = len(support2.split('Name of donor'))-1
    record['support_other'] = support2b
    record['support_other_COUNT'] = len(support2b.split('Name of donor'))-1
    record['family_lobbying'] = famlobbying.replace('members engaged in lobbying the public sector on behalf of a third party or client','')
    record['family'] = fam.replace('members employed and paid from parliamentary expenses','')
    record['giftsUK'] = gifts3.replace(', benefits and hospitality from UK sources','')
    record['giftsUK_COUNT'] = len(gifts3.replace(', benefits and hospitality from UK sources','').split('Name of donor'))-1
    record['giftsNonUK'] = gifts5.replace('and benefits from sources outside the UK','')
    record['giftsNonUK_COUNT'] = len(gifts5.replace('and benefits from sources outside the UK','').split('Name of donor'))-1
    record['URL'] = baseurl+latestregister.replace('contents.htm',mpurl).replace('part1','')
    record['date'] = latestregister.replace('contents.htm','').replace('part1','').replace('/','')
    scraperwiki.sql.save(['URL'], record)


